In [8]:
import os
import gzip

import pandas as pd

from tqdm import tqdm
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
es = Elasticsearch()

data_dir = "clean"

In [3]:
example = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", id=1, body=example)
print(res['result'])

updated


In [4]:
dir_names = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]

In [5]:
def load_data(directory):
    dir_names = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    
    dfs = {}
    for d_name in tqdm(dir_names, "Directory"):
        filenames = [f for f in os.listdir(f"{directory}/{d_name}") if os.path.isfile(os.path.join(f"{directory}/{d_name}", f))]
        station = []
        for f_name in filenames:
            file = gzip.open(f"{directory}/{d_name}/{f_name}", "r")
            station.append(pd.read_csv(file, sep=";"))
        dfs[d_name] = pd.concat(station)
    
    return dfs

In [6]:
dfs = load_data(data_dir)

Directory: 100%|██████████| 25/25 [02:55<00:00,  7.04s/it]


In [13]:
def gendata(dfs):
    id = 0
    for name, df in dfs.items():
        yield {
            "_id": id,
            "_index": name[4:].lower(),
            "_type": "document",
            "doc": df.to_dict(),
        }
        id += 1

bulk(es, gendata(dfs))

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))